In [1]:
! pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

#**Push-Ups**

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 Pose model
model = YOLO('yolov8n-pose.pt')  # or 'yolov8x-pose.pt' for a larger model
video_path = '/content/pushup.mp4'  # Path to your input video
out_path = 'results_pushUp.mp4'  # Path to save the output video

def calculate_angle(p1, p2, p3):
    """ Calculate the angle between three points. """
    a = np.array(p1)
    b = np.array(p2)
    c = np.array(p3)
    ab = a - b
    bc = c - b
    angle = np.arccos(np.clip(np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc)), -1.0, 1.0))
    return np.degrees(angle)

def draw_keypoints(frame, keypoints, indices):
    """ Draw specific keypoints on the frame. """
    for i in indices:
        point = keypoints[i]
        cv2.circle(frame, tuple(map(int, point)), 10, (255, 0, 0), -1)  # Blue points for selected keypoints

def draw_connections(frame, keypoints, connections):
    """ Draw lines between specific keypoints to visualize connections. """
    for start_idx, end_idx in connections:
        start_point = tuple(map(int, keypoints[start_idx]))
        end_point = tuple(map(int, keypoints[end_idx]))
        cv2.line(frame, start_point, end_point, (0, 255, 0), 5)  # Green lines for connections

def main():
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Initialize VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' for .mp4 format
    out = cv2.VideoWriter(out_path, fourcc, fps, (640, 384))  # Output video resolution

    push_up_count = 0
    push_up_state = "up"

    # Define connections based on specific pose estimation keypoints (shoulder, elbow, and wrist)
    connections = [
        (5, 7),  # Shoulder to Elbow
        (7, 9),  # Elbow to Wrist
    ]

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Resize the frame to (640, 384)
        frame_resized = cv2.resize(frame, (640, 384))

        # Preprocess frame for YOLOv8 Pose
        results = model(frame_resized)

        print(results[0].keypoints)

        # Extract keypoints
        keypoints = np.array(results[0].keypoints.xy.tolist()).reshape(-1, 2)  # Assuming keypoints are in the first result
        if len(keypoints) >= 10:
            # Extract keypoints (assuming order is shoulder, elbow, and wrist)
            shoulder = keypoints[5]
            elbow = keypoints[7]
            wrist = keypoints[9]

            angle = calculate_angle(shoulder, elbow, wrist)

            # Determine push-up state based on angle
            if angle < 80:  # Adjust angle threshold based on your observation
                if push_up_state == "up":
                    push_up_count += 1
                    push_up_state = "down"
            elif angle > 150:  # Adjust angle threshold based on your observation
                if push_up_state == "down":
                    push_up_state = "up"

            # Draw only the shoulder, elbow, and wrist keypoints
            draw_keypoints(frame_resized, keypoints, [5, 7, 9])

            # Draw connections between shoulder, elbow, and wrist
            draw_connections(frame_resized, keypoints, connections)

            # Display the angle on the frame
            cv2.putText(frame_resized, f"Angle: {angle:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)

        # Display the count and state on the frame
        cv2.putText(frame_resized, f"Push-ups: {push_up_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame_resized, f"State: {push_up_state}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2, cv2.LINE_AA)

        # Write the frame to the output video
        out.write(frame_resized)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


100%|██████████| 6.52M/6.52M [00:00<00:00, 103MB/s]


Streaming output truncated to the last 5000 lines.
         [ 98.2861, 243.2795],
         [ 78.9965, 213.0467]]], device='cuda:0')
xyn: tensor([[[0.7881, 0.3247],
         [0.7968, 0.3108],
         [0.7900, 0.2881],
         [0.0000, 0.0000],
         [0.7587, 0.2089],
         [0.7195, 0.2646],
         [0.6715, 0.2549],
         [0.6593, 0.4499],
         [0.6275, 0.5404],
         [0.6521, 0.6768],
         [0.6249, 0.7576],
         [0.4674, 0.4028],
         [0.4349, 0.3557],
         [0.3651, 0.5605],
         [0.3059, 0.4855],
         [0.1536, 0.6335],
         [0.1234, 0.5548]]], device='cuda:0')

0: 384x640 1 person, 10.2ms
Speed: 1.4ms preprocess, 10.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
ultralytics.engine.results.Keypoints object with attributes:

conf: tensor([[0.9672, 0.5848, 0.9730, 0.1030, 0.9552, 0.9685, 0.9988, 0.8326, 0.9971, 0.7162, 0.9719, 0.9914, 0.9976, 0.9513, 0.9847, 0.7868, 0.8723]], device='cuda:0')
data: tensor([[[5.0302e+02,

#**Bicep**

In [7]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 Pose model
model = YOLO('yolov8n-pose.pt')  # or 'yolov8x-pose.pt' for a larger model
video_path = '/content/Bicep 4.mp4'  # Path to your input video
out_path = 'results_bicep_curls.mp4'  # Path to save the output video

def calculate_angle(p1, p2, p3):
    """ Calculate the angle between three points. """
    a = np.array(p1)
    b = np.array(p2)
    c = np.array(p3)
    ab = a - b
    bc = c - b
    angle = np.arccos(np.clip(np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc)), -1.0, 1.0))
    return np.degrees(angle)

def draw_keypoints(frame, keypoints, indices):
    """ Draw specific keypoints on the frame. """
    for i in indices:
        point = keypoints[i]
        cv2.circle(frame, tuple(map(int, point)), 10, (255, 0, 0), -1)  # Blue points for selected keypoints

def draw_connections(frame, keypoints, connections):
    """ Draw lines between specific keypoints to visualize connections. """
    for start_idx, end_idx in connections:
        start_point = tuple(map(int, keypoints[start_idx]))
        end_point = tuple(map(int, keypoints[end_idx]))
        cv2.line(frame, start_point, end_point, (0, 255, 0), 5)  # Green lines for connections

def main():
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get original video dimensions
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Initialize VideoWriter with original dimensions
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' for .mp4 format
    out = cv2.VideoWriter(out_path, fourcc, fps, (width, height))  # Use original dimensions

    curl_count = 0
    curl_state = "down"

    # Define connections based on specific pose estimation keypoints (shoulder, elbow, wrist, and hip)
    connections = [
        (5, 7),  # Shoulder to Elbow
        (7, 9),  # Elbow to Wrist
        (5, 11)  # Shoulder to Hip (to show body alignment)
    ]

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess frame for YOLOv8 Pose
        results = model(frame)

        # Extract keypoints
        keypoints = np.array(results[0].keypoints.xy.tolist()).reshape(-1, 2)  # Assuming keypoints are in the first result

        if len(keypoints) >= 12:  # Ensure enough keypoints are detected
            # Extract keypoints (shoulder, elbow, wrist, and hip)
            shoulder = keypoints[5]
            elbow = keypoints[7]
            wrist = keypoints[9]
            hip = keypoints[11]

            arm_angle = calculate_angle(shoulder, elbow, wrist)
            body_arm_angle = calculate_angle(hip, shoulder, elbow)
            correct_postion = True

            # Determine curl state based on arm angle, only count if body-arm angle is less than or equal to 20 degrees
            if body_arm_angle <= 20:
                if arm_angle < 60:  # Adjust angle threshold based on your observation
                    if curl_state == "down":
                        curl_count += 1
                        curl_state = "up"
                elif arm_angle > 140:  # Adjust angle threshold based on your observation
                    if curl_state == "up":
                        curl_state = "down"
            else:
                correct_postion = False

            # Draw keypoints and connections
            draw_keypoints(frame, keypoints, [5, 7, 9, 11])
            draw_connections(frame, keypoints, connections)

            # Display the angles on the frame
            cv2.putText(frame, f"Arm: {arm_angle:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, f"Bo: {body_arm_angle:.2f}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Display the count and state on the frame
        cv2.putText(frame, f"Curls: {curl_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, f"State: {curl_state}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, f"Correct: {'Yes' if correct_postion else 'No'}", (30, 140), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if correct_postion else (0, 0, 255), 2, cv2.LINE_AA)

        # Write the frame to the output video
        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()



0: 384x640 1 person, 10.6ms
Speed: 2.6ms preprocess, 10.6ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 8.5ms
Speed: 2.8ms preprocess, 8.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7.3ms
Speed: 1.8ms preprocess, 7.3ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7.1ms
Speed: 1.6ms preprocess, 7.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7.6ms
Speed: 1.3ms preprocess, 7.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7.9ms
Speed: 1.2ms preprocess, 7.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7.9ms
Speed: 1.4ms preprocess, 7.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7.6ms
Speed: 1.6ms preprocess, 7.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 38

<ipython-input-7-3ba716cb5731>:17: RuntimeWarning: invalid value encountered in scalar divide
  angle = np.arccos(np.clip(np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc)), -1.0, 1.0))



0: 384x640 (no detections), 8.7ms
Speed: 1.3ms preprocess, 8.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.9ms
Speed: 1.3ms preprocess, 9.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.0ms
Speed: 1.9ms preprocess, 10.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.5ms
Speed: 1.4ms preprocess, 10.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.8ms
Speed: 2.7ms preprocess, 12.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.2ms
Speed: 1.4ms preprocess, 11.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.4ms
Speed: 2.2ms preprocess, 10.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.8ms
Speed: 2.0ms preprocess, 12.8ms infer

#**Squats**

In [4]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 Pose model
model = YOLO('yolov8n-pose.pt')  # or 'yolov8x-pose.pt' for a larger model
video_path = '/content/squat.mp4'  # Path to your input video
out_path = 'results_squats.mp4'  # Path to save the output video

def calculate_angle(p1, p2, p3):
    """ Calculate the angle between three points. """
    a = np.array(p1)
    b = np.array(p2)
    c = np.array(p3)
    ab = a - b
    bc = c - b
    angle = np.arccos(np.clip(np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc)), -1.0, 1.0))
    return np.degrees(angle)

def draw_keypoints(frame, keypoints, indices):
    """ Draw specific keypoints on the frame. """
    for i in indices:
        point = keypoints[i]
        cv2.circle(frame, tuple(map(int, point)), 7, (255, 0, 0), -1)  # Blue points for selected keypoints

def draw_connections(frame, keypoints, connections, color):
    """ Draw lines between specific keypoints to visualize connections. """
    for start_idx, end_idx in connections:
        start_point = tuple(map(int, keypoints[start_idx]))
        end_point = tuple(map(int, keypoints[end_idx]))
        cv2.line(frame, start_point, end_point, color, 3)  # Color-coded lines for connections

def main():
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get original video dimensions
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Initialize VideoWriter with original dimensions
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' for .mp4 format
    out = cv2.VideoWriter(out_path, fourcc, fps, (width, height))  # Use original dimensions

    squat_count = 0
    squat_state = "up"

    # Define connections for keypoints: (shoulder, hip, knee, ankle)
    connections = [
        (11, 13),  # Hip to Knee
        (13, 15),  # Knee to Ankle
        (12, 11),  # Shoulder to Hip
        (5, 11),   # Neck to Hip
    ]

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess frame for YOLOv8 Pose
        results = model(frame)

        # Extract keypoints
        keypoints = np.array(results[0].keypoints.xy.tolist()).reshape(-1, 2)  # Assuming keypoints are in the first result
        if len(keypoints) >= 16:  # Ensure enough keypoints are detected
            # Extract keypoints (shoulder, hip, knee, ankle)
            shoulder = keypoints[12]  # Shoulder keypoint
            hip = keypoints[11]       # Hip keypoint
            hip2 = keypoints[12]      # Shoulder keypoint
            knee = keypoints[13]      # Knee keypoint
            ankle = keypoints[15]     # Ankle keypoint

            # Calculate angles
            knee_angle = calculate_angle(hip, knee, ankle)
            shoulder_hip_knee_angle = calculate_angle(shoulder, hip, knee)
            hip_hip_knee_angle = calculate_angle(hip2, hip, knee)

            # Determine squat state based on knee angle
            if knee_angle < 125:  # Adjusted angle threshold for squat down position
                if squat_state == "up":
                    squat_count += 1
                    squat_state = "down"
            elif knee_angle > 160:  # Adjusted angle threshold for squat up position
                if squat_state == "down":
                    squat_state = "up"

            # Check leg position based on hip-hip-knee angle
            correct_leg_position = True
            if hip_hip_knee_angle < 90 :
                correct_leg_position = False  # Leg should be more extended in the "up" position

            # Draw keypoints and connections with specific colors for angles
            draw_keypoints(frame, keypoints, [12, 11, 13, 15])
            draw_connections(frame, keypoints, [(11, 13), (13, 15)], (0, 255, 255))  # Yellow for knee angle
            # draw_connections(frame, keypoints, [(12, 11)], (0, 255, 0))  # Green for shoulder-hip-knee angle
            draw_connections(frame, keypoints, [(12,11)], (255, 0, 0))  # Blue for hip-hip-knee angle

            # Display the angles in a white box at the upper right corner
            cv2.rectangle(frame, (width - 220, 10), (width - 10, 150), (255, 255, 255), -1)
            cv2.putText(frame, f"KA: {knee_angle:.2f}", (width - 210, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, f"HHK: {hip_hip_knee_angle:.2f}", (width - 210, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            # cv2.putText(frame, f"SHK: {shoulder_hip_knee_angle:.2f}", (width - 210, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            # Display the correct positions for legs and back
            cv2.putText(frame, f"Legs: {'Yes' if correct_leg_position else 'No'}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if correct_leg_position else (0, 0, 255), 2, cv2.LINE_AA)
            # cv2.putText(frame, f"Back: {'Yes' if correct_back_position else 'No'}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if correct_back_position else (0, 0, 255), 2, cv2.LINE_AA)

        # Display the count and state on the frame
        cv2.putText(frame, f"Squats: {squat_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, f"State: {squat_state}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2, cv2.LINE_AA)

        # Write the frame to the output video
        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()



0: 384x640 1 person, 8.0ms
Speed: 1.1ms preprocess, 8.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 8.4ms
Speed: 1.4ms preprocess, 8.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7.6ms
Speed: 1.6ms preprocess, 7.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7.3ms
Speed: 1.4ms preprocess, 7.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7.5ms
Speed: 2.5ms preprocess, 7.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7.3ms
Speed: 2.4ms preprocess, 7.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7.4ms
Speed: 1.8ms preprocess, 7.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7.5ms
Speed: 1.8ms preprocess, 7.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x